In [5]:
import sys
from pathlib import Path

project_root = Path.cwd().parent  # or Path().resolve().parent
sys.path.insert(0, str(project_root))

from src.evaluate import evaluate

In [6]:
#example
generations = ["i have bought an owl, but now it wants to fly away", "my grandma broke my camera, and she did not even notice"]
grounds = ["an owl was bought, and it wants to fly to the grocery store", "it wasnt noticed but my dad, bought a camera"]
p, r, F1 = evaluate(generations, grounds)
print(F1)

tensor([0.6863, 0.5159])
